In [ ]:
import os
import time
from PIL import Image
import gc
import psutil
import functools
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import numpy as np
from loguru import logger
import torch.distributed as dist
from torch import optim
from trainers.base_trainer import BaseTrainer
from utils.ema import EMA
from utils.model_helper import import_model, loss_fn
from utils.vis_helper import visualize_point_clouds_3d
from utils.eval_helper import compute_NLL_metric 
from utils import model_helper, exp_helper, data_helper
from utils.data_helper import normalize_point_clouds
from utils.diffusion_pvd import DiffusionDiscretized
from utils.diffusion_continuous import make_diffusion, DiffusionBase
from utils.checker import *
from utils import utils
from matplotlib import pyplot as plt
import third_party.pvcnn.functional as pvcnn_fn
from timeit import default_timer as timer
from torch.optim import Adam as FusedAdam
from torch.cuda.amp import autocast, GradScaler
from trainers import common_fun_prior_train

In [26]:
from utils.diffusion import make_beta_schedule
beta_start = 0.1
beta_end = 20.0
num_steps = 100
mode = 'linear'

betas = make_beta_schedule(
            mode, beta_start, beta_end, num_steps).numpy()
betas[:10]

array([0.1       , 0.3010101 , 0.5020202 , 0.7030303 , 0.9040404 ,
       1.10505051, 1.30606061, 1.50707071, 1.70808081, 1.90909091])

In [27]:
def _generate_base_constants(betas, diffusion_steps):
    """
    Generates torch tensors with basic constants for all timesteps.
    """
    betas_np = betas  # self._generate_betas_from_continuous_fun(diffusion_steps)

    alphas_np = 1.0 - betas_np
    alphas_cumprod = alpha_bars_np = np.cumprod(alphas_np)
    snr = 1.0 / (1 - alphas_cumprod) - 1

    # posterior variances only make sense for t>1, hence the array is short by 1
    betas_post_np = betas_np[1:] * \
        (1.0 - alpha_bars_np[:-1]) / (1.0 - alpha_bars_np[1:])
    # we add beta_post_2 to the beginning of both beta arrays, since this is used as final decoder variance and
    # requires special treatment (as in diffusion paper)
    betas_post_init_np = np.append(betas_post_np[0], betas_post_np)
    #betas_init_np = np.append(betas_post_np[0], betas_np[1:])

    betas_init = torch.from_numpy(betas_np).float().cuda()
    snr = torch.from_numpy(snr).float().cuda()
    alphas = torch.from_numpy(alphas_np).float().cuda()
    alpha_bars = torch.from_numpy(alpha_bars_np).float().cuda()
    betas_post_init = torch.from_numpy(betas_post_init_np).float().cuda()

    return betas_init, alphas, alpha_bars, betas_post_init, snr

betas_init, alphas, alpha_bars, betas_post_init, snr = _generate_base_constants(betas, num_steps)

In [35]:
print(type(betas_init))
# print(alphas)
# print(alpha_bars)
print(betas_post_init.shape)
# print(snr)

<class 'torch.Tensor'>
torch.Size([100])


In [40]:
def get_p_log_scales(betas_init, betas_post_init, timestep, stddev_type):
    """
    Grab log std devs. of backward denoising process p, if we decide to fix them.
    """
    if stddev_type == 'beta':
        # use diffusion variances, except for t=1, for which we use posterior variance beta_post_2
        index = torch.tensor([timestep - 1], dtype=torch.long)  # Tạo tensor chỉ số
        return 0.5 * torch.log(torch.gather(betas_init, 0, index))
    elif stddev_type == 'beta_post':
        # use diffusion posterior variances, except for t=1, for which there is no posterior, so we use beta_post_2
        index = torch.tensor([timestep - 1], dtype=torch.long)  # Tạo tensor chỉ số
        print(index)
        temp = torch.gather(betas_post_init, 0, index)
        print(temp)
        return 0.5 * torch.log(torch.gather(betas_post_init, 0, index))
    elif stddev_type == 'learn':
        return None
    else:
        raise ValueError('Unknown stddev_type: {}'.format(stddev_type))

# Ví dụ kiểm tra
betas_init = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5])  # Mô phỏng một tensor beta
betas_post_init = torch.tensor([0.05, 0.15, 0.25, 0.35, 0.45])  # Mô phỏng một tensor beta_post

# Gọi hàm
result = get_p_log_scales(betas_init, betas_post_init, 3, 'beta_post')
print(result)
print(betas_post_init)

tensor([2])
tensor([0.2500])
tensor([-0.6931])
tensor([0.0500, 0.1500, 0.2500, 0.3500, 0.4500])


In [45]:
def get_mixing_component(x_noisy, time_step, enabled, _alpha_bars):
    size = x_noisy.size()
    alpha_bars = torch.gather(_alpha_bars, 0, time_step-1)
    one_minus_alpha_bars = utils.view4D(torch.sqrt(1.0 - alpha_bars), size)
    return x_noisy * one_minus_alpha_bars

alpha_bars = torch.tensor([0.9, 0.8, 0.7, 0.6, 0.5])
get_mixing_component(torch.tensor([1, 2, 3, 4]), 3, True, alpha_bars)


tensor([[[[0.5477, 1.0954, 1.6432, 2.1909]]],


        [[[0.5477, 1.0954, 1.6432, 2.1909]]],


        [[[0.5477, 1.0954, 1.6432, 2.1909]]],


        [[[0.5477, 1.0954, 1.6432, 2.1909]]]])


In [ ]:
@torch.no_grad()
def run_ddim(self, model, num_samples, shape, temp=1.0, enable_autocast=False, 
            is_image=True, prior_var=1.0, conditional_input=None, ddim_step=100, skip_type='uniform', kappa=1.0,
            clip_feat=None, grid_emb=None, x_noisy=None, dae_index=-1):
    model.eval()
    x_noisy_size = [num_samples] + shape
    x_noisy = torch.randn(
        size=x_noisy_size, device='cuda') if x_noisy is None else x_noisy.cuda()
    output_list = []
    S = ddim_step
    if skip_type == 'uniform':
        c = (self._diffusion_steps - 1.0) / (S - 1.0)
        list_tau = [np.floor(i * c) for i in range(S)]
        list_tau = [int(s) for s in list_tau]
    user_defined_steps = sorted(list(list_tau), reverse=True)
    T_user = len(user_defined_steps)

    Alpha_bar = self._alpha_bars
    for i, t in enumerate(user_defined_steps):
        tau = t 
        timestep = torch.ones(
            num_samples, dtype=torch.int64, device='cuda') * (t+1)
        fixed_log_scales = self.get_p_log_scales(
                timestep=timestep, stddev_type=self._denoising_stddevs)
        mixing_component = self.get_mixing_component(
                x_noisy, timestep, enabled=model.mixed_prediction)
        with autocast(enable_autocast):
            pred_logits = model(
                    x=x_noisy, t=timestep.float(), condition_input=condition_input)
            logits = utils.get_mixed_prediction(
                    model.mixed_prediction, pred_logits, model.mixing_logit, mixing_component)

        


In [18]:
diffusion_steps = 1000
S = 100
num_samples = 2

c = (diffusion_steps - 1.0) / (S - 1.0)
list_tau = [np.floor(i * c) for i in range(S)]
list_tau = [int(s) for s in list_tau]
user_defined_steps = sorted(list(list_tau), reverse=True)
print(list_tau[:10])
print(list_tau[-10:])
print(user_defined_steps)

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
[908, 918, 928, 938, 948, 958, 968, 978, 988, 999]
[999, 988, 978, 968, 958, 948, 938, 928, 918, 908, 898, 888, 877, 867, 857, 847, 837, 827, 817, 807, 797, 787, 777, 766, 756, 746, 736, 726, 716, 706, 696, 686, 676, 666, 655, 645, 635, 625, 615, 605, 595, 585, 575, 565, 555, 544, 534, 524, 514, 504, 494, 484, 474, 464, 454, 444, 433, 423, 413, 403, 393, 383, 373, 363, 353, 343, 333, 322, 312, 302, 292, 282, 272, 262, 252, 242, 232, 222, 211, 201, 191, 181, 171, 161, 151, 141, 131, 121, 111, 100, 90, 80, 70, 60, 50, 40, 30, 20, 10, 0]


In [20]:
for i, t in enumerate(user_defined_steps):
    tau = t 
    timestep = torch.ones(
        num_samples, dtype=torch.int64) * (t+1)
    

tensor([1000, 1000])
tensor([989, 989])
tensor([979, 979])
tensor([969, 969])
tensor([959, 959])
tensor([949, 949])
tensor([939, 939])
tensor([929, 929])
tensor([919, 919])
tensor([909, 909])
tensor([899, 899])
tensor([889, 889])
tensor([878, 878])
tensor([868, 868])
tensor([858, 858])
tensor([848, 848])
tensor([838, 838])
tensor([828, 828])
tensor([818, 818])
tensor([808, 808])
tensor([798, 798])
tensor([788, 788])
tensor([778, 778])
tensor([767, 767])
tensor([757, 757])
tensor([747, 747])
tensor([737, 737])
tensor([727, 727])
tensor([717, 717])
tensor([707, 707])
tensor([697, 697])
tensor([687, 687])
tensor([677, 677])
tensor([667, 667])
tensor([656, 656])
tensor([646, 646])
tensor([636, 636])
tensor([626, 626])
tensor([616, 616])
tensor([606, 606])
tensor([596, 596])
tensor([586, 586])
tensor([576, 576])
tensor([566, 566])
tensor([556, 556])
tensor([545, 545])
tensor([535, 535])
tensor([525, 525])
tensor([515, 515])
tensor([505, 505])
tensor([495, 495])
tensor([485, 485])
tensor([47

In [4]:
num_samples = 2 
shape = [2,1]
size = [num_samples] + shape
x_noisy = torch.randn(size=size)
print(x_noisy.size())

torch.Size([2, 2, 1])


In [ ]:
@torch.no_grad()
def generate_samples_vada(shape, dae, diffusion, vae, num_samples, 
                            enable_autocast, ode_eps=0.00001, ode_solver_tol=1e-5,
                            ode_sample=False, prior_var-1.0, temp=1.0, vae_temp=1.0,
                            noise=None, need_denoise=False, ddim_step=0, clip_feat=None):
    output = {}
    if not ode_sample:
        assert isinstance(diffusion, DiffusionDiscretized), 'Regular sampling requires disc. diffusion!'
        assert noise is None, 'Noise is not used in ancestral sampling.'
        total_step = diffusion._diffusion_steps
        if ddim_step > 0:
            eps, eps_list = diffusion.run_ddim(dae, )


In [48]:
import importlib
def build_model(model_name):
    model_lib = importlib.import_module(model_name)
    model = model_lib.Model()
    return model

vae = build_model('models.vae_adain')

TypeError: Model.__init__() missing 1 required positional argument: 'args'

In [ ]:

DAE = import_model('models.latent_points_ada_localprior.PVCNN2Prior')
model = 
class Trainer(BaseTrainer):
    is_diffusion = 0

    def train_iter(self, data, *args, **kwargs):
        device = 'cuda'
        input_dim = 3 
        loss_type = 'mse'
        dae = DAE(args, num_input_channels, self.cfg).to(device)
        vae = build_model('models.vae_adain').to(device)
        diffusion = DiffusionDiscretized()
        dae_optimizer = optim.Adam(param_dict_dae,
                                   lr=args.learning_rate_dae,
                                   betas=(cfgopt.beta1, cfgopt.beta2),
                                   weight_decay=cfgopt.weight_decay)
        vae_optimizer = optim.Adam(vae.parameters(),
                                   lr=args.learning_rate_min_vae,
                                   betas=(cfgopt.beta1, cfgopt.beta2),
                                   weight_decay=cfgopt.weight_decay)
        args = self.cfg.sde
        num_total_iter = cfg.trainer.epochs * len(train_loader)
        dae_sn_calculator = self.dae_sn_calculator
        vae_sn_calculator = self.vae_sn_calculator
        grad_scalar = self.grad_scalar

        global_step = step = kwargs.get('step', None)
        no_update = kwargs.get('no_update', False)

        # update_lr
        warmup_iters = len(self.train_loader) * args.warmup_epochs
        utils.update_lr(args, global_step, warmup_iters,
                        dae_optimizer, vae_optimizer)

        # input
        tr_pts = data['tr_points'].to(device)  # (B, Npoints, 3)
        # the noisy points, used in trainers/voxel2pts.py and trainers/voxel2pts_ada.py
        inputs = data['input_pts'].to(device) if 'input_pts' in data else None
        B = batch_size = tr_pts.size(0)

        # optimize vae params
        vae_optimizer.zero_grad()
        output = self.compute_loss_vae(tr_pts, global_step, inputs=inputs)

        if args.train_dae:
            eps = output['eps'].detach() #B, D, N, 1
            dae_optimizer.zero_grad()
            with autocast():
                noise_p = torch.rand(size=eps.size(), device=device)
                



        

In [57]:
import torch
import torch.nn.functional as F

# Hàm point_tracking như đã định nghĩa
def point_tracking(F0, F1, handle_points, handle_points_init, args):
    with torch.no_grad():
        _, _, max_r, max_c = F0.shape
        for i in range(len(handle_points)):
            pi0, pi = handle_points_init[i], handle_points[i]
            f0 = F0[:, :, int(pi0[0]), int(pi0[1])]

            

            r1, r2 = max(0, int(pi[0]) - args.r_p), min(max_r, int(pi[0]) + args.r_p + 1)
            c1, c2 = max(0, int(pi[1]) - args.r_p), min(max_c, int(pi[1]) + args.r_p + 1)
            
            
            F1_neighbor = F1[:, :, r1:r2, c1:c2]
            all_dist = (f0.unsqueeze(dim=-1).unsqueeze(dim=-1) - F1_neighbor).abs().sum(dim=1)
            all_dist = all_dist.squeeze(dim=0)
            row, col = divmod(all_dist.argmin().item(), all_dist.shape[-1])
            if i == 0:
                print(f0)
                print(pi[1])
                print(c1)
                print(c2)
                print(row)
                print(col)
            
            handle_points[i][0] = r1 + row
            handle_points[i][1] = c1 + col
        return handle_points

# Định nghĩa class args với tham số r_p (bán kính tìm kiếm)
class Args:
    def __init__(self, r_p):
        self.r_p = r_p

# Giả lập dữ liệu đầu vào
B, C, H, W = 1, 3, 100, 100  # batch size, số channels, chiều cao và chiều rộng của feature maps

# Feature maps F0 và F1 ngẫu nhiên
F0 = torch.rand(B, C, H, W)
F1 = torch.rand(B, C, H, W)

# Danh sách các điểm điều khiển ban đầu và hiện tại (giả lập)
handle_points_init = [[50, 50], [30, 30]]  # ví dụ các điểm ở tọa độ (50, 50) và (30, 30)
handle_points = [[55, 55], [35, 35]]       # ví dụ vị trí ban đầu tại (55, 55) và (35, 35)

# Tạo đối tượng args với bán kính tìm kiếm r_p
args = Args(r_p=5)

# Gọi hàm point_tracking
updated_points = point_tracking(F0, F1, handle_points, handle_points_init, args)

# In ra kết quả
print("Updated handle points:", updated_points)

tensor([[0.0743, 0.5052, 0.4140]])
55
50
61
7
6
Updated handle points: [[57, 56], [34, 33]]


In [3]:
import torch
temp = torch.tensor([0.0000, 1.7900, 0.0000])
temp.repeat(2048, 1).shape

torch.Size([2048, 3])